In [2]:
# import pandas as pd
import csv
import numpy as np
import os
from os.path import join as pjoin
from glob import iglob

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
#!/usr/bin/env python
from __future__ import division

import argparse
import glob
import os
import random
import signal
import time

import torch

import distributed
from models import data_loader, model_builder
from models.data_loader import load_dataset
from models.model_builder import ExtSummarizer
from models.trainer_ext import build_trainer
from others.logging import logger, init_logger

model_flags = ['hidden_size', 'ff_size', 'heads', 'inter_layers', 'encoder', 'ff_actv', 'use_interval', 'rnn_size']


In [4]:
root_path = '/data/ksb/'
bert_root_path = pjoin(root_path, 'BertSum/PreSumm')
bert_model_dir = pjoin(bert_root_path, 'models')

data_dir = pjoin(root_path, 'cnn-dailymail/finished_files')
three_data_dir = pjoin(root_path, 'three-mat')
three_data_test = pjoin(three_data_dir, 'test')


#### Loss function 비교  

*Trained Model parameter 필요*


In [7]:
def get_cos_similarity(inputs, sents):
    tfidf_vectorizer = TfidfVectorizer()

    cos_similarity_list = []
    for input_, sent_ in zip(inputs, sents):
        try:
            tfidf_matrix = tfidf_vectorizer.fit_transform([input_, sent_])

            similarity = cosine_similarity(tfidf_matrix[0] , tfidf_matrix[1])[0][0]
        except ValueError:
            similarity = 0.0
            
        cos_similarity_list.append(similarity)

    return cos_similarity_list

In [44]:
import jsonlines
import json

data_list = []
for data_p in iglob(pjoin(three_data_test, '**.json'), recursive=False):
    
    with open(data_p,'r',encoding='utf-8') as f:
        data = json.load(f)
        data_list.append(data)

### Origin candidate set

In [45]:
from rouge import Rouge
rouge = Rouge()

In [46]:
doc_sim_list = []
ref_sim_list = []

min_doc_sim_list = []
max_doc_sim_list = []

doc_rouge_list = []
ref_rouge_list = []

max_ref_rouge_list = []
min_ref_rouge_list = []

for data in data_list:
    candidates = data['candidates']
    article = data['article']
    abstract = data["abstract"]
    
    sim_list = [np.mean(get_cos_similarity(article, cand[0])) for cand in candidates]
    doc_sim_list += sim_list
    min_doc_sim_list += [min(sim_list)]
    max_doc_sim_list += [max(sim_list)]
    
    ref_sim_list += [np.mean(get_cos_similarity(abstract, cand[0])) for cand in candidates]
    
    doc_rouge_list +=[rouge.get_scores('\n'.join(cand[0]), '\n'.join(article))[0]['rouge-l']['f'] for cand in candidates]
    
    rouge_list = [rouge.get_scores('\n'.join(cand[0]), '\n'.join(abstract))[0]['rouge-l']['f'] for cand in candidates]
    ref_rouge_list += rouge_list
    min_ref_rouge_list += [min(rouge_list)]
    max_ref_rouge_list += [max(rouge_list)]

In [47]:
print("Mean cosine similarity between document and summaries : {}".format(round(np.mean(doc_sim_list), 4)))
print("Mean cosine similarity between reference and summaries : {}\n".format(round(np.mean(ref_sim_list), 4)))

print("Min cosine similarity between document and summaries : {}".format(round(np.mean(min_doc_sim_list), 4)))
print("Max cosine similarity between document and summaries : {}".format(round(np.mean(max_doc_sim_list), 4)))

Mean cosine similarity between document and summaries : 0.2746
Mean cosine similarity between reference and summaries : 0.1834

Min cosine similarity between document and summaries : 0.1434
Max cosine similarity between document and summaries : 0.4457


In [48]:
print("Mean Rouge score between document and summaries : {}".format(round(np.mean(doc_rouge_list), 4)))
print("Mean Rouge score between reference and summaries : {}\n".format(round(np.mean(ref_rouge_list), 4)))

print("Min Rouge score between reference and summaries : {}".format(round(np.mean(min_ref_rouge_list), 4)))
print("Max Rouge score between reference and summaries : {}".format(round(np.mean(max_ref_rouge_list), 4)))

Mean Rouge score between document and summaries : 0.3288
Mean Rouge score between reference and summaries : 0.4156

Min Rouge score between reference and summaries : 0.358
Max Rouge score between reference and summaries : 0.4719


In [49]:
from transformers import BertTokenizer

def bert_encode(x, max_len=-1):
    tok = BertTokenizer.from_pretrained('bert-base-uncased', verbose=False)
    cls_token_id = tok.cls_token_id
    sep_token_id = tok.sep_token_id

    _ids = tok.encode(x, add_special_tokens=False)
    ids = [cls_token_id] # [CLS]
    if max_len > 0:
        ids.extend(_ids[:max_len - 2])
    else:
        ids.extend(_ids[:512 - 2])
    ids.append(sep_token_id) # [SEP], meaning end of sentence
    return ids

In [50]:
def bert_decode(x):
    tok = BertTokenizer.from_pretrained('bert-base-uncased', verbose=False)

    result = tok.decode(x, skip_special_tokens=True)
    return result

In [28]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

n_gram = 3


In [29]:
def rewrite(cand_set, sent_id, window_id, tok, model, n_gram='trigram', tok_max_length=512, max_length=5):
        
    if n_gram =='trigram':
        n_gram = 3
    elif n_gram =='4-gram':
        n_gram = 4
    elif n_gram == '5-gram':
        n_gram = 5
    else:
        n_gram = None
    
    def _map_sent_id(cand_set):
        mapping_ = []
        for i, cand in enumerate(cand_set) :
            summ_ = []
            for j, s in enumerate(cand):
                idx = sum([len(prev) for prev in cand_set[:i]]) + j
                summ_.append((idx, s))
            
            mapping_.append(summ_)
            
        return mapping_
    
    def _get_sent_fromId(cand_set, idx):
        dscored = [(i, j, sent[1]) for i, cand in enumerate(cand_set) for j, sent in enumerate(cand) if sent[0]==idx]
        return dscored[0]
    
    mapped_cand_set = _map_sent_id(cand_set)
    set_idx, sent_idx, sent = _get_sent_fromId(mapped_cand_set, sent_id)
    
    sent_tok = sent.split()
    n_gram = n_gram if n_gram is not None else len(sent_tok)
    
    origin_phase_txt = ' '.join(sent_tok[window_id : window_id + n_gram])    
    print(origin_phase_txt)
    
    bart_tokenized = tok([origin_phase_txt], max_length=tok_max_length, return_tensors='pt')
    bart_summarized = model.generate(bart_tokenized['input_ids'], num_beams=4, max_length=max_length, early_stopping=True)
    result = [tok.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in bart_summarized]
    
    print(result)
        
    sent_tok[window_id:window_id + n_gram] = result
    cand_set[set_idx][sent_idx] = ' '.join(sent_tok)
    
    return cand_set

In [30]:
def map_sent_id(cand_set):
    mapping_ = []
    for i, cand in enumerate(cand_set) :
        summ_ = []
        for j, s in enumerate(cand):
            idx = sum([len(prev) for prev in cand_set[:i]]) + j
            summ_.append((idx, s))
            
        mapping_.append(summ_)
            
    return mapping_

In [31]:
def mask_rewrite(cand_set, sent_id, window_id, tok, model, n_gram='trigram', tok_max_length=512, max_length=5):
        
    if n_gram =='trigram':
        n_gram = 3
    elif n_gram =='4-gram':
        n_gram = 4
    elif n_gram == '5-gram':
        n_gram = 5
    else:
        n_gram = None
    
    def _map_sent_id(cand_set):
        mapping_ = []
        for i, cand in enumerate(cand_set) :
            summ_ = []
            for j, s in enumerate(cand):
                idx = sum([len(prev) for prev in cand_set[:i]]) + j
                summ_.append((idx, s))
            
            mapping_.append(summ_)
            
        return mapping_
    
    def _get_sent_fromId(cand_set, idx):
        dscored = [(i, j, sent[1]) for i, cand in enumerate(cand_set) for j, sent in enumerate(cand) if sent[0]==idx]
        return dscored[0]
    
    mapped_cand_set = _map_sent_id(cand_set)
    set_idx, sent_idx, sent = _get_sent_fromId(mapped_cand_set, sent_id)
    
    sent_tok = sent.split()
    sent_tok[window_id:window_id + n_gram] = ['<mask>']
    n_gram = n_gram if n_gram is not None else len(sent_tok)
    
    origin_phase_txt = sent    
    print(' '.join(sent_tok))
    
    bart_tokenized = tok(origin_phase_txt, return_tensors='pt')
    bart_summarized = model.generate(bart_tokenized['input_ids'], max_length=max_length)
    result = tok.batch_decode(bart_summarized, skip_special_tokens=True)
    
    print(result)
        
    sent_tok[window_id:window_id + n_gram] = result
    cand_set[set_idx][sent_idx] = result[0]
    
    return cand_set

In [51]:
## Phase Rewrite
ph_re_doc_sims = []
ph_re_ref_sims = []

min_ph_re_doc_sims = []
max_ph_re_doc_sims = []

## Sentence Rewrite
se_re_doc_sims = []
se_re_ref_sims = []

min_se_re_doc_sims = []
max_se_re_doc_sims = []

## Phase masking & Rewriting
ph_mr_doc_sims = []
ph_mr_ref_sims = []

min_ph_mr_doc_sims = []
max_ph_mr_doc_sims = []

In [52]:
## Phase Rewrite
ph_re_doc_rouges = []
ph_re_ref_rouges = []

min_ph_re_doc_rouges = []
max_ph_re_doc_rouges = []

## Sentence Rewrite
se_re_doc_rouges = []
se_re_ref_rouges = []

min_se_re_doc_rouges = []
max_se_re_doc_rouges = []

## Phase masking & Rewriting
ph_mr_doc_rouges = []
ph_mr_ref_rouges = []

min_ph_mr_doc_rouges = []
max_ph_mr_doc_rouges = []

In [53]:
import copy
import jsonlines

In [54]:
new_data_path = pjoin(three_data_dir,'test.jsonl')

In [55]:
with open(new_data_path, 'w', encoding='utf-8') as f:
    writer = jsonlines.Writer(f)
    
    for data in data_list:
        candidates = data['candidates']
        article = data['article']
        abstract = data["abstract"]

        scores = []
        summaries = [cand[0] for cand in candidates]
        encoded_cand_set = [[bert_encode(s, 180) for s in summ] for summ in summaries]

        for i, summary in enumerate(summaries): # candidate summary set 개수
            for j, sent in enumerate(summary): # sentence 개수

                ext_tok = summaries[i][j].split()
                window = len(ext_tok) // n_gram

                sent_id = sum([len(prev) for prev in summaries[:i]]) + j

                for w in range(window):
                    s = rouge.get_scores(' '.join(ext_tok[w:w+n_gram]), '\n'.join(abstract))[0]['rouge-l']['f']

                    score = (sent_id, w, ' '.join(ext_tok[w:w+n_gram]), s)
                    scores.append(score)
        dscored = sorted(scores,key=lambda x: x[-1], reverse=True)
        dscored = [sc for sc in dscored if sc[-1]==0.0]


        ## Rewrite Phase
        ph_re_summaries = copy.deepcopy(summaries)

        for sent_id, w, ph, _ in dscored:

            mapped_cand_set = map_sent_id(encoded_cand_set)
            tmp = [(i, j, sent[1]) for i, cand in enumerate(mapped_cand_set) 
                               for j, sent in enumerate(cand) if sent[0]==sent_id]
            _, _, sent = tmp[0]

            ph_re_summaries = rewrite(cand_set=ph_re_summaries, sent_id=sent_id, \
                                         window_id=w, tok=tokenizer, model=model, n_gram='trigram',
                                        tok_max_length=512, max_length=7)

        sim_list = [np.mean(get_cos_similarity(article, cand)) for cand in ph_re_summaries]
        ph_re_doc_sims += sim_list
        min_ph_re_doc_sims += [min(sim_list)]
        max_ph_re_doc_sims += [max(sim_list)]
        ph_re_ref_sims = [np.mean(get_cos_similarity(abstract, cand)) for cand in ph_re_summaries]

        rouge_list = [rouge.get_scores('\n'.join(cand), '\n'.join(article))[0]['rouge-l']['f'] for cand in ph_re_summaries]
        ph_re_doc_rouges += rouge_list
        min_ph_re_doc_rouges += [min(rouge_list)]
        max_ph_re_doc_rouges += [max(rouge_list)]
        ph_re_ref_rouges = [rouge.get_scores('\n'.join(cand), '\n'.join(abstract))[0]['rouge-l']['f'] for cand in ph_re_summaries]

        ## Rewrite Sentence

        se_re_summaries = copy.deepcopy(summaries)

        for sent_id, w, ph, s in dscored:
            window_id = 0

            mapped_cand_set = map_sent_id(encoded_cand_set)
            tmp = [(i, j, sent[1]) for i, cand in enumerate(mapped_cand_set) 
                               for j, sent in enumerate(cand) if sent[0]==sent_id]
            _, _, sent = tmp[0]

            se_re_summaries = rewrite(cand_set=se_re_summaries, sent_id=sent_id, \
                                         window_id=window_id, tok=tokenizer, model=model, n_gram=None,
                                        tok_max_length=512, max_length=len(sent))

        sim_list = [np.mean(get_cos_similarity(article, cand)) for cand in se_re_summaries]
        se_re_doc_sims += sim_list
        min_se_re_doc_sims += [min(sim_list)]
        max_se_re_doc_sims += [max(sim_list)]
#         se_re_ref_sims += [np.mean(get_cos_similarity(abstract, cand)) for cand in se_re_summaries]

        rouge_list = [rouge.get_scores('\n'.join(cand), '\n'.join(article))[0]['rouge-l']['f'] for cand in se_re_summaries]
        se_re_doc_rouges += rouge_list
        min_se_re_doc_rouges += [min(rouge_list)]
        max_se_re_doc_rouges += [max(rouge_list)]
#         se_re_ref_rouges += [rouge.get_scores('\n'.join(cand), '\n'.join(abstract))[0]['rouge-l']['f'] for cand in se_re_summaries]


        ## Phase masking and Rewriting

        ph_mr_summaries = copy.deepcopy(summaries)

        for sent_id, w, ph, _ in dscored:
            mapped_cand_set = map_sent_id(encoded_cand_set)
            tmp = [(i, j, sent[1]) for i, cand in enumerate(mapped_cand_set) 
                               for j, sent in enumerate(cand) if sent[0]==sent_id]
            _, _, sent = tmp[0]

            ph_mr_summaries = mask_rewrite(cand_set=ph_mr_summaries, sent_id=sent_id, \
                                         window_id=window_id, tok=tokenizer, model=model, n_gram='trigram',
                                        tok_max_length=512, max_length=len(sent))

        sim_list = [np.mean(get_cos_similarity(article, cand)) for cand in ph_mr_summaries]
        ph_mr_doc_sims += sim_list
        min_ph_mr_doc_sims += [min(sim_list)]
        max_ph_mr_doc_sims += [max(sim_list)]
#         ph_mr_ref_sims += [np.mean(get_cos_similarity(abstract, cand)) for cand in ph_mr_summaries]

        rouge_list = [rouge.get_scores('\n'.join(cand), '\n'.join(article))[0]['rouge-l']['f'] for cand in ph_mr_summaries]
        ph_mr_doc_rouges += rouge_list
        min_ph_mr_doc_rouges += [min(rouge_list)]
        max_ph_mr_doc_rouges += [max(rouge_list)]
#         ph_mr_ref_rouges += [rouge.get_scores('\n'.join(cand), '\n'.join(abstract))[0]['rouge-l']['f'] for cand in ph_mr_summaries]

        new_data = {'article':article, 'candidates':candidates, 'abstract':abstract,
            'phase-rewrite ': ph_re_summaries, 'sentence_rewrite': se_re_summaries, 'phase-masking-rewriting':ph_mr_summaries}
        writer.write(new_data)
        

club tijuana star
['club tijuana star']
conjured memories luis
['conjured memories l']
memories l suarez
['memories l su']
was not booked
['was not booked.']
not booked. by
['not booked. by']
( left )
['( left)']
left) bites the
['left) bites the']
venezuelan icon sank
['venezuelan']
was not booked
['was not booked.']
not booked. by
['not booked. by']
venezuelan icon sank
['venezuelan']
was not booked
['was not booked.']
not booked. by
['not booked. by']
club tijuana star juan arango conjured memories luis suarez in his team 's 4-3 defeat by monterrey in the mexican league - but it was not through prodigious scoring .
["Club tijuana star juan arango conjured memories luis suarez in his team 's 4-3 defeat by monterrey in the mexican league - but it was"]
Club tijuana star juan arango conjured memories luis suarez in his team 's 4-3 defeat by monterrey in the mexican league - but it was
["Club tijuana star juan arango conjured memories luis suarez in his team 's 4-3 defeat by monterrey i

['format has been changed']
has been changed
['has been changed']
meet bud dodson
['meet bud dodson']
has been living
['has been living']
has been living
['has been living']
unusual format has
['unusual format has']
format has been
['format has been changed']
has been changed
['has been changed']
she came across
['she came across']
when she stopped
['when she stopped']
has been living
['has been living']
the unusual format has been captured in a series of photographs by visual journalist anna erickson .
['The unusual format has been captured in a series of photographs by visual journalist anna er']
The unusual format has been captured in a series of photographs by visual journalist anna er
['The unusual format has been captured in a series of photographs by visual journalist anna er']
The unusual format has been captured in a series of photographs by visual journalist anna er
['The unusual format has been captured in a series of photographs by visual journalist anna er']
meet bud dodso

['messi trained alongside compatriot javier mascherano on th Thursday. Both are expected to rejoin the rest of the squad on']
<mask> with compatriot javier mascherano on thursday ahead of sunday 's trip to celta vigo .
["lionel messi trained with compatriot javier mascherano on th Thursday ahead of sunday's trip to c"]
<mask> both of his country 's friendlies against el salvador and ecuador over the international break .
["messi sat out both of his country's friendlies against el salvador and ecuador over the"]
<mask> both of his country's friendlies against el salvador and ecuador over the
["messi sat out both of his country's friendlies against el salvador and ecuador over the weekend"]
alec stewart admitted
['alec stewart']
stewart he has
['stewart he']
contact with the
['contact with the']
roy hodgson is
['roy hodg']
hodg launching roses
['hodg launching roses']
launching roses 2015
['launching roses 2015']
roses 2015 ,
['roses 2015 ,']
2015 , the
['2015 , the']
main problem is
['m

['it was widely believed that the new ecb regime led by chief executive tom harrison and chairman colin graves would waste little time in appointing from a short list of former england captains michael vaughan']
it was widely believed that the new ecb regime led by chief executive tom harrison and chairman colin graves would waste little time in appointing from a short list of former england captains michael vaughan
['it was widely believed that the new ecb regime led by chief executive tom harrison and chairman colin graves would waste little time in appointing from a short list of former england captains michael vaughan']
it was widely believed that the new ecb regime led by chief executive tom harrison and chairman colin graves would waste little time in appointing from a short list of former england captains michael vaughan
['it was widely believed that the new ecb regime led by chief executive tom harrison and chairman colin graves would waste little time in appointing from a shor

['it was widely believed that the new ecb regime led by chief executive tom harrison and chairman colin graves would waste little time in appointing from a short list of former england captains michael vaughan']
<mask> believed that the new ecb regime led by chief executive tom harrison and chairman colin graves would waste little time in appointing from a short list of former england captains michael vaughan
['it was widely believed that the new ecb regime led by chief executive tom harrison and chairman colin graves would waste little time in appointing from a short list of former england captains michael vaughan']
<mask> believed that the new ecb regime led by chief executive tom harrison and chairman colin graves would waste little time in appointing from a short list of former england captains michael vaughan
['it was widely believed that the new ecb regime led by chief executive tom harrison and chairman colin graves would waste little time in appointing from a short list of form

...

["an estimated 957,000 shoppers in the us alone ordered apple watches on friday. This popularity surpassed expectations - even apple 's"]
in a leaked memo from the firm 's angela ahrendts , the retail chief said that customers wo n't be able to buy an apple watch in store ` through the month of may ' due to ` high global interest combined with our initial supply . '
["in a leaked memo from the firm 's angela ahrendts , the retail chief said that customers wo n't be able to buy an apple watch in store ` through the month of may ' due to ' high global interest combined with our initial supply . '"]
an estimated 957,000 shoppers in the us alone ordered apple watches on friday and this popularity surpassed expectations - even apple 's .
["an estimated 957,000 shoppers in the us alone ordered apple watches on friday. This popularity surpassed expectations - even apple 's"]
an estimated 957,000 shoppers in the us alone ordered apple watches on friday. This popularity surpassed expectations -

["in a leaked memo from the firm's angela ahrendts the retail chief wrote : ` due to high global interest combined with our initial supply, we are only taking orders online right now -lsb- and -rsb- we expect this to continue through the month of may' The memo was seen by rhiannon williams at rh"]
<mask> shoppers in the us alone ordered apple watches on friday and this popularity surpassed expectations - even apple 's .
["an estimated 957,000 shoppers in the us alone ordered apple watches on friday. This popularity surpassed expectations - even apple's"]
<mask> shoppers in the us alone ordered apple watches on friday. This popularity surpassed expectations - even apple's
["an estimated 957,000 shoppers in the us alone ordered apple watches on friday. This popularity surpassed expectations - even apple's."]
<mask> shoppers in the us alone ordered apple watches on friday. This popularity surpassed expectations - even apple's.
["an estimated 957,000 shoppers in the us alone ordered apple 

["The monster vegetable is so big it had been nicknamed the ` fat little girl ' in the village where it was grown. The monster vegetable weighs more than a three-year-old child , the people 's daily online"]
The monster vegetable is so big it had been nicknamed the ` fat little girl ' in the village where it was grown. The monster vegetable weighs more than a three-year-old child , the people 's daily online
["The monster vegetable is so big it had been nicknamed the ` fat little girl ' in the village where it was grown. The monster vegetable weighs more than a three-year-old child , the people 's daily online"]
The monster vegetable is so big it had been nicknamed the ` fat little girl ' in the village where it was grown. The monster vegetable weighs more than a three-year-old child , the people 's daily online
["The monster vegetable is so big it had been nicknamed the ` fat little girl ' in the village where it was grown. The monster vegetable weighs more than a three-year-old child

["The monster vegetable is so big it had been nicknamed the ` fat little girl' in the village where it was grown. The monster vegetable weighs more than a three-year-old child, the people's daily online newspaper"]
<mask> , which weighs more than a three-year-old child , is so big it had been nicknamed the ` fat little girl ' in the village where it was grown .
["The monster vegetable weighs more than a three-year-old child. It is so big it had been nicknamed the ` fat little girl'in the village where it was grown."]
<mask> weighs more than a three-year-old child. It is so big it had been nicknamed the ` fat little girl'in the village where it was grown.
["The monster vegetable weighs more than a three-year-old child. It is so big it had been nicknamed the ` fat little girl' in the village where it was grown."]
<mask> weighs more than a three-year-old child. It is so big it had been nicknamed the ` fat little girl' in the village where it was grown.
["The monster vegetable weighs more 

ValueError: Connection error, and we cannot find the requested files in the cached path. Please try again or make sure your Internet connection is on.

Rewrite Phase

In [66]:
print("Mean cosine similarity between document and summaries : {}".format(round(np.mean(ph_re_doc_sims), 4)))
# print("Mean cosine similarity between reference and summaries : {}\n".format(round(np.mean(ref_sim_list), 4)))

print("Min cosine similarity between document and summaries : {}".format(round(np.mean(min_ph_re_doc_sims), 4)))
print("Max cosine similarity between document and summaries : {}\n".format(round(np.mean(max_ph_re_doc_sims), 4)))

print("Mean ROUGE score between document and summaries : {}".format(round(np.mean(ph_re_doc_rouges), 4)))
# print("Mean cosine similarity between reference and summaries : {}\n".format(round(np.mean(ph_mr_ref_rouges), 4)))

# print("Min ROUGE score between document and summaries : {}".format(round(np.mean(min_ph_re_doc_rouges), 4)))
# print("Max ROUGE score between document and summaries : {}".format(round(np.mean(max_ph_re_doc_rouges), 4)))

Mean cosine similarity between document and summaries : 0.2648
Min cosine similarity between document and summaries : 0.1456
Max cosine similarity between document and summaries : 0.4089

Mean ROUGE score between document and summaries : 0.3242


Rewrite Sentence

In [67]:
print("Mean cosine similarity between document and summaries : {}".format(round(np.mean(se_re_doc_sims), 4)))
# print("Mean cosine similarity between reference and summaries : {}\n".format(round(np.mean(ref_sim_list), 4)))

print("Min cosine similarity between document and summaries : {}".format(round(np.mean(min_se_re_doc_sims), 4)))
print("Max cosine similarity between document and summaries : {}\n".format(round(np.mean(max_se_re_doc_sims), 4)))

print("Mean ROUGE score between document and summaries : {}".format(round(np.mean(se_re_doc_rouges), 4)))
# print("Mean cosine similarity between reference and summaries : {}\n".format(round(np.mean(ph_mr_ref_rouges), 4)))

# print("Min ROUGE score between document and summaries : {}".format(round(np.mean(min_se_re_doc_rouges), 4)))
# print("Max ROUGE score between document and summaries : {}".format(round(np.mean(max_se_re_doc_rouges), 4)))

Mean cosine similarity between document and summaries : 0.257
Min cosine similarity between document and summaries : 0.1418
Max cosine similarity between document and summaries : 0.3952

Mean ROUGE score between document and summaries : 0.3118


Phase masking and Rewriting

In [68]:
print("Mean cosine similarity between document and summaries : {}".format(round(np.mean(ph_mr_doc_sims), 4)))
# print("Mean cosine similarity between reference and summaries : {}\n".format(round(np.mean(ref_sim_list), 4)))

print("Min cosine similarity between document and summaries : {}".format(round(np.mean(min_ph_mr_doc_sims), 4)))
print("Max cosine similarity between document and summaries : {}\n".format(round(np.mean(max_ph_mr_doc_sims), 4)))

print("Mean ROUGE score between document and summaries : {}".format(round(np.mean(ph_mr_doc_rouges), 4)))
# print("Mean cosine similarity between reference and summaries : {}\n".format(round(np.mean(ph_mr_ref_rouges), 4)))

# print("Min ROUGE score between document and summaries : {}".format(round(np.mean(min_ph_mr_doc_rouges), 4)))
# print("Max ROUGE score between document and summaries : {}".format(round(np.mean(max_ph_mr_doc_rouges), 4)))

Mean cosine similarity between document and summaries : 0.257
Min cosine similarity between document and summaries : 0.142
Max cosine similarity between document and summaries : 0.395

Mean ROUGE score between document and summaries : 0.3017
